In [1]:
import ee
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
#import contextily as ctx
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob

from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import plotly
import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import geopandas as gpd

import pandas as pd
from datetime import timedelta, date
import geopandas as gpd
from uuid import uuid4


## Load heatwaves and get initial control dates

In [2]:
daily_max_dic = {}

for state in ['California', 'Oregon', 'Washington', 'British Columbia']:
            
    daily_max_files = glob.glob(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/{state}/max*')
    
    temp = dd.read_csv(daily_max_files)
    temp = temp.compute()
    daily_max_dic[state] = temp
    

full_df = pd.concat(daily_max_dic.values())

full_df['datetime'] = pd.to_datetime(full_df[['day', 'month', 'year']])
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

full_df['location'] = full_df['county'] + ', ' + full_df['state']
full_df.drop_duplicates(['location', 'datetime'], inplace=True)

full_df = full_df[(full_df['month'] != 10)]
full_df = full_df[(full_df['month'] != 4)]

full_df['pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [12]:
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['temp_pct'] = full_df.groupby('location')['temperature_2m'].transform(lambda x: x.rank(pct=True))

In [3]:
#heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwaves_90.csv')
heatwaves = pd.read_csv(f'/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/heatwave_clean_90_v1.csv')
heatwaves = heatwaves[heatwaves['heatwave'] == 1]
#full_df = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/daily_max/max_temps/max_temps.csv')

heatwaves['datetime'] = pd.to_datetime(heatwaves['datetime'])

In [4]:
#identify and remove heatwaves that begin in may
may_hw = heatwaves.groupby('heat_ID').apply(lambda x: 5 in x['month'].unique()).reset_index()
may_hw = may_hw[may_hw[0] == True]
heatwaves = heatwaves[~heatwaves['heat_ID'].isin(may_hw['heat_ID'].values)]

In [5]:
#set initial control day to one week prior
heatwaves['control_day'] = heatwaves['datetime'] - timedelta(days=7)

In [6]:
#get day of week of each control day
heatwaves['day_of_week'] = [x.weekday() for x in heatwaves['datetime']]

In [7]:
heatwaves = heatwaves[heatwaves['year'] >= 2020]

In [8]:
heatwaves.reset_index(inplace=True)

In [9]:
full_df.reset_index(inplace=True)

In [ ]:
# condition that if control day is in heat wave days (the dictionary corresponding to the county)
# and that the temperature of the control day is not 
 

for index, row in heatwaves.iterrows():
    
    i = 0
   
    while (full_df[(full_df['datetime'] == heatwaves.iloc[index, -2]) & (full_df['location'] == row['location'])]['temp_pct'].values[0]>= 0.8):

        if (i == 0) | (i == 1):
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] - timedelta(days=7)
            print(heatwaves.iloc[index, -2])
            print(i)

        if (i == 2):
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] + timedelta(days=28)
            print(heatwaves.iloc[index, -2])

        if (i == 3):
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] + timedelta(days=7)
            print(heatwaves.iloc[index, -2])

        if i >=4:
            print(row['location'])
            print(row['control_day'])
            heatwaves.iloc[index, -2] = heatwaves.iloc[index, -2] - timedelta(days=7)
            print(heatwaves.iloc[index, -2])
            print(i)
        
        i+=1


In [14]:
heatwaves

,index,Unnamed: 0.1,Unnamed: 0,county,day,month,year,temperature_2m,state,datetime,datetime_ord,location,pct,level_0,heatwave,heat_ID,Heat Wave Duration_x,Heat Wave Duration_y,control_day,day_of_week
0,43,559,5550,San Francisco,2,6,2020,26.468729,California,2020-06-02,737578,"San Francisco, California",0.901961,46.0,1.0,006dc71f-3793-4e79-b7ec-ba2e8a67458d,2.0,2.0,2020-05-19,1
1,44,131,935,Contra Costa,3,6,2020,37.984781,California,2020-06-03,737579,"Contra Costa, California",0.965453,49.0,1.0,422efa81-d9df-4376-9009-05456a3592f1,2.0,2.0,2020-05-20,2
2,45,560,5555,San Francisco,3,6,2020,27.366281,California,2020-06-03,737579,"San Francisco, California",0.936508,47.0,1.0,006dc71f-3793-4e79-b7ec-ba2e8a67458d,NaN,NaN,2020-05-20,2
3,46,786,8173,Tuolumne,3,6,2020,37.574625,California,2020-06-03,737579,"Tuolumne, California",0.902894,36.0,1.0,0ff908b9-c2e8-4be8-a7e4-da968e2b6432,2.0,2.0,2020-05-20,2
4,47,503,5093,Sacramento,3,6,2020,39.387903,California,2020-06-03,737579,"Sacramento, California",0.901027,38.0,1.0,de02d61c-a4f3-472b-a18b-337d512c5225,2.0,2.0,2020-05-20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,7564,4208,2359,Josephine,10,9,2022,37.008951,Oregon,2022-09-10,738408,"Josephine, Oregon",0.956116,90.0,1.0,4c80f805-0b12-4d8f-8861-928d8563b600,NaN,NaN,2022-09-03,5
7496,7565,4379,4053,Tillamook,10,9,2022,33.444498,Oregon,2022-09-10,738408,"Tillamook, Oregon",0.976657,90.0,1.0,86249ac7-d27d-4931-b151-b8bcfdf5daf7,NaN,NaN,2022-09-03,5
7497,7566,4258,2821,Lane,10,9,2022,36.538248,Oregon,2022-09-10,738408,"Lane, Oregon",0.980392,89.0,1.0,05df6599-c345-435a-a93c-9b9de4cc6f6d,NaN,NaN,2022-09-03,5
7498,7567,4291,3129,Linn,10,9,2022,36.508951,Oregon,2022-09-10,738408,"Linn, Oregon",0.980392,92.0,1.0,743d83f5-dbd4-4170-8581-e04ac9ce4f6b,NaN,NaN,2022-09-03,5


In [20]:
heatwaves.iloc[:, 3:]

,county,day,month,year,temperature_2m,state,datetime,datetime_ord,location,pct,level_0,heatwave,heat_ID,Heat Wave Duration_x,Heat Wave Duration_y,control_day,day_of_week
0,San Francisco,2,6,2020,26.468729,California,2020-06-02,737578,"San Francisco, California",0.901961,46.0,1.0,006dc71f-3793-4e79-b7ec-ba2e8a67458d,2.0,2.0,2020-05-19,1
1,Contra Costa,3,6,2020,37.984781,California,2020-06-03,737579,"Contra Costa, California",0.965453,49.0,1.0,422efa81-d9df-4376-9009-05456a3592f1,2.0,2.0,2020-05-20,2
2,San Francisco,3,6,2020,27.366281,California,2020-06-03,737579,"San Francisco, California",0.936508,47.0,1.0,006dc71f-3793-4e79-b7ec-ba2e8a67458d,NaN,NaN,2020-05-20,2
3,Tuolumne,3,6,2020,37.574625,California,2020-06-03,737579,"Tuolumne, California",0.902894,36.0,1.0,0ff908b9-c2e8-4be8-a7e4-da968e2b6432,2.0,2.0,2020-05-20,2
4,Sacramento,3,6,2020,39.387903,California,2020-06-03,737579,"Sacramento, California",0.901027,38.0,1.0,de02d61c-a4f3-472b-a18b-337d512c5225,2.0,2.0,2020-05-20,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,Josephine,10,9,2022,37.008951,Oregon,2022-09-10,738408,"Josephine, Oregon",0.956116,90.0,1.0,4c80f805-0b12-4d8f-8861-928d8563b600,NaN,NaN,2022-09-03,5
7496,Tillamook,10,9,2022,33.444498,Oregon,2022-09-10,738408,"Tillamook, Oregon",0.976657,90.0,1.0,86249ac7-d27d-4931-b151-b8bcfdf5daf7,NaN,NaN,2022-09-03,5
7497,Lane,10,9,2022,36.538248,Oregon,2022-09-10,738408,"Lane, Oregon",0.980392,89.0,1.0,05df6599-c345-435a-a93c-9b9de4cc6f6d,NaN,NaN,2022-09-03,5
7498,Linn,10,9,2022,36.508951,Oregon,2022-09-10,738408,"Linn, Oregon",0.980392,92.0,1.0,743d83f5-dbd4-4170-8581-e04ac9ce4f6b,NaN,NaN,2022-09-03,5


In [21]:
heatwaves = heatwaves.iloc[:, 3:]

In [22]:
#export.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_90_v2.csv')
heatwaves.to_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave_control/heatwave_control_clean_90_v1.csv')